In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pysabr import Hagan2002LognormalSABR
from pysabr import hagan_2002_lognormal_sabr as sabr
import xgboost as xgb
from math import sqrt
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split

In [ ]:
# build time series model for sabr parameters 

In [193]:
def create_features(df):
    df=df.copy()
    df['rhol1']=df.shift(1)['rho']
    df['volvoll1']=df.shift(1)['volvol']
    df['Forward Swap Ratel1']=df.shift(1)['Forward Swap Rate']
    df['rhol2']=df.shift(2)['rho']
    df['volvoll2']=df.shift(2)['volvol']
    df['Forward Swap Ratel2']=df.shift(2)['Forward Swap Rate']
    df['rholrm2']=df['rho'].shift(1).rolling(2).mean()
    df['volvollrm2']=df['volvol'].shift(1).rolling(2).mean()
    df['Forward Swap Ratelrm2']=df['Forward Swap Rate'].shift(1).rolling(2).mean()
    df['hour'] = df.index.hour
    df['dayofweek'] = df.index.dayofweek
    df['quarter'] = df.index.quarter
    df['month'] = df.index.month
    df['year'] = df.index.year
    df['dayofyear'] = df.index.dayofyear
    df['dayofmonth'] = df.index.day
    df['weekofyear'] = df.index.isocalendar().week
    return df
def create_featuresa(df):
    df=df.copy()
    df['alphal1']=df.shift(1)['alpha']
    df['Forward Swap Ratel1']=df.shift(1)['Forward Swap Rate']
    df['alphal2']=df.shift(2)['alpha']
    df['Forward Swap Ratel2']=df.shift(2)['Forward Swap Rate']
    df['alphalrm2']=df['alpha'].shift(1).rolling(2).mean()
    df['Forward Swap Ratelrm2']=df['Forward Swap Rate'].shift(1).rolling(2).mean()
    df['hour'] = df.index.hour
    df['dayofweek'] = df.index.dayofweek
    df['quarter'] = df.index.quarter
    df['month'] = df.index.month
    df['year'] = df.index.year
    df['dayofyear'] = df.index.dayofyear
    df['dayofmonth'] = df.index.day
    df['weekofyear'] = df.index.isocalendar().week
    df['alphal3']=df.shift(3)['alpha']
    return df

In [2]:
swaption=pd.read_csv("sabr.csv") #sabr is swaption with sabr parameters calibrated
swaption['Date']=pd.to_datetime(swaption['Date'],format='%Y-%m-%d')
swaption=swaption.drop('Unnamed: 0',axis=1)
swaption_u=swaption.copy()
swaption_u=swaption_u.drop(['error','abs(e_size)>0.005'],axis=1)
swaption_u.set_index('Date',inplace=True)
swaption_u

In [ ]:
datasets = {name: group for name, group in swaption_u.groupby(['Expiry', 'Tenor'])}

In [ ]:
unique_combos=pd.read_csv('unique_combo.csv')
unique_combos=unique_combos.drop('Unnamed: 0',axis=1)

In [238]:
for i in range(len(unique_combos)):
    a,b=unique_combos.loc[i,:]
    df=datasets[a,b]
    #fit alpha
    dfa=df[df['Strike']=='atm'].drop_duplicates('Forward Swap Rate',keep='first')[['alpha','Forward Swap Rate','Vols','sabrvol']]
    dfa=create_featuresa(dfa)
    xa=dfa.drop(['alpha','Vols','sabrvol'],axis=1)
    ya=dfa[['alpha']]
    x_train, x_testa, y_train, y_testa = train_test_split(xa, ya, test_size=1 - 0.8,shuffle=False)#, random_state = seed, shuffle=False)
    reg = xgb.XGBRegressor(base_score=0.5, booster='gbtree',    
                           n_estimators=1000,
                           early_stopping_rounds=70,
                           objective='reg:linear',
                           max_depth=3,
                           learning_rate=0.009)
    reg.fit(x_train, y_train,
        eval_set=[(x_train, y_train), (x_testa, y_testa)],
        verbose=100)
    filename = f'model/{a}_{b}_alphamodel.json'
    reg.save_model(filename)
    #fit others
    dfo=df[df['Strike']=='atm'].drop_duplicates('Forward Swap Rate',keep='first')[['rho','volvol','Forward Swap Rate','Vols','sabrvol']]
    dfo=create_features(dfo)
    x=dfo.drop(['rho','volvol','Vols','sabrvol'],axis=1)
    y=dfo[['rho','volvol']]
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=1 - 0.8,shuffle=False)#, random_state = seed, shuffle=False)
    reg = xgb.XGBRegressor(base_score=0.5, booster='gbtree',    
                           n_estimators=1000,
                           early_stopping_rounds=70,
                           objective='reg:linear',
                           max_depth=3,
                           learning_rate=0.009)
    reg.fit(x_train, y_train,
        eval_set=[(x_train, y_train), (x_test, y_test)],
        verbose=100)
    filename = f'model/{a}_{b}_rhovolmodel.json'
    reg.save_model(filename)
    
    

[0]	validation_0-rmse:0.49015	validation_1-rmse:0.48397
[100]	validation_0-rmse:0.19881	validation_1-rmse:0.19269
[200]	validation_0-rmse:0.08071	validation_1-rmse:0.07604
[300]	validation_0-rmse:0.03280	validation_1-rmse:0.03029


/Users/unimportantnotes/opt/anaconda3/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [12:05:45] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


[400]	validation_0-rmse:0.01335	validation_1-rmse:0.01237
[500]	validation_0-rmse:0.00545	validation_1-rmse:0.00715
[600]	validation_0-rmse:0.00224	validation_1-rmse:0.00532
[700]	validation_0-rmse:0.00096	validation_1-rmse:0.00478
[800]	validation_0-rmse:0.00048	validation_1-rmse:0.00448
[900]	validation_0-rmse:0.00034	validation_1-rmse:0.00437
[999]	validation_0-rmse:0.00031	validation_1-rmse:0.00433
[0]	validation_0-rmse:0.38345	validation_1-rmse:0.35678
[100]	validation_0-rmse:0.15615	validation_1-rmse:0.13372


/Users/unimportantnotes/opt/anaconda3/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [12:05:45] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


[200]	validation_0-rmse:0.06411	validation_1-rmse:0.05921
[300]	validation_0-rmse:0.02701	validation_1-rmse:0.03881
[394]	validation_0-rmse:0.01304	validation_1-rmse:0.03929
[0]	validation_0-rmse:0.48954	validation_1-rmse:0.48161
[100]	validation_0-rmse:0.19859	validation_1-rmse:0.19077
[200]	validation_0-rmse:0.08063	validation_1-rmse:0.07572


...

[200]	validation_0-rmse:0.06348	validation_1-rmse:0.03929
[300]	validation_0-rmse:0.02782	validation_1-rmse:0.01989
[400]	validation_0-rmse:0.01453	validation_1-rmse:0.01852
[485]	validation_0-rmse:0.01063	validation_1-rmse:0.01907
[0]	validation_0-rmse:0.49159	validation_1-rmse:0.48763
[100]	validation_0-rmse:0.19937	validation_1-rmse:0.19542
[200]	validation_0-rmse:0.08088	validation_1-rmse:0.07695
[300]	validation_0-rmse:0.03286	validation_1-rmse:0.02988
[400]	validation_0-rmse:0.01336	validation_1-rmse:0.01193


/Users/unimportantnotes/opt/anaconda3/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [12:06:58] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


[500]	validation_0-rmse:0.00544	validation_1-rmse:0.00485
[600]	validation_0-rmse:0.00223	validation_1-rmse:0.00207
[700]	validation_0-rmse:0.00093	validation_1-rmse:0.00116
[800]	validation_0-rmse:0.00042	validation_1-rmse:0.00095
[900]	validation_0-rmse:0.00025	validation_1-rmse:0.00093
[961]	validation_0-rmse:0.00021	validation_1-rmse:0.00093
[0]	validation_0-rmse:0.36952	validation_1-rmse:0.28662
[100]	validation_0-rmse:0.15104	validation_1-rmse:0.11261
[200]	validation_0-rmse:0.06281	validation_1-rmse:0.05680


/Users/unimportantnotes/opt/anaconda3/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [12:06:58] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


[300]	validation_0-rmse:0.02770	validation_1-rmse:0.03900
[400]	validation_0-rmse:0.01475	validation_1-rmse:0.03709
[464]	validation_0-rmse:0.01164	validation_1-rmse:0.03774
